![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

# AQL Geospatial Tutorial

<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/AqlGeospatialTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the previous parts of this AQL tutorial series, we have already covered a number of topics:
* [Part 1: CRUD](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb) 
* [Part 2: Limit, Sort, Filter](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlPart2Tutorial.ipynb)
* [Part 3: Join](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlJoinTutorial.ipynb)
* [Part 3: Graph Traversals](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlTraversalTutorial.ipynb)

In this part we learn how AQL supports geospatial queries.
Geospatial coordinates consisting of a latitude and longitude value can be stored either as two separate attributes, or as a single attribute in the form of an array with both numeric values. ArangoDB can index such coordinates for fast geospatial queries.

Recall, that in addition to the python interface, you can interact with ArangoDB using its web interface to manage collections and execute the queries.

# Setup 

Before getting started with ArangoDB we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

Create the temporary database:

In [6]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AQLGeoSpatial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login["dbName"]] 

Requesting new temp credentials.
Temp database ready to use.


In [7]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUTmdxdiz8dt8kqgmur8we81n
Password: TUT8m9cc75nu08bko6mj89sum
Database: TUTjfhw6szvz0hg1sprwe0ayw


Feel free to use to above URL to checkout the UI!

##  Locations Data

Let us create a collection with some filming locations for Games of Thrones.

![locations](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/Locations_Map.png?raw=1)

In [8]:
# Create the Locaions Collection 
db.createCollection(name="Locations")

ArangoDB collection name: Locations, id: 18020733, type: document, status: loaded

In [9]:
insert_query = """
LET places = [
    { "name": "Dragonstone", "coordinate": [ 55.167801, -6.815096 ] },
    { "name": "King's Landing", "coordinate": [ 42.639752, 18.110189 ] },
    { "name": "The Red Keep", "coordinate": [ 35.896447, 14.446442 ] },
    { "name": "Yunkai", "coordinate": [ 31.046642, -7.129532 ] },
    { "name": "Astapor", "coordinate": [ 31.50974, -9.774249 ] },
    { "name": "Winterfell", "coordinate": [ 54.368321, -5.581312 ] },
    { "name": "Vaes Dothrak", "coordinate": [ 54.16776, -6.096125 ] },
    { "name": "Beyond the wall", "coordinate": [ 64.265473, -21.094093 ] }
]


FOR place IN places
    INSERT place INTO Locations
"""

db.AQLQuery(insert_query)

As before let us check the `Locations` collection:

In [10]:
all_locations_names = """
FOR p IN Locations
    RETURN p.name
"""

query_result = db.AQLQuery(all_locations_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Dragonstone

King's Landing

The Red Keep

Yunkai

Astapor

Winterfell

Vaes Dothrak

Beyond the wall



# Geospatial Index

To query based on coordinates, a [geo index](https://www.arangodb.com/docs/stable/indexing-geo.html) is required. It determines which fields contain the latitude and longitude values.

In [11]:
db["Locations"].ensureGeoIndex(["coordinate"])

# Find nearby Locations

A `FOR` loop is used to iterate over the results of a function call to NEAR() to find the n closest coordinates to a reference point, and return the documents with the nearby locations. The default for n is 100, which means 100 documents are returned at most, the closest matches first.

In below example, the limit is set to 3. The origin (the reference point) is a coordinate somewhere downtown in Dublin, Ireland:

In [12]:
near_locations_names = """
FOR loc IN NEAR(Locations, 53.35, -6.26, 3)
    RETURN {
        name: loc.name,
        latitude: loc.coordinate[0],
        longitude: loc.coordinate[1]
    }
"""

query_result = db.AQLQuery(near_locations_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()    

{'name': 'Vaes Dothrak', 'latitude': 54.16776, 'longitude': -6.096125}

{'name': 'Winterfell', 'latitude': 54.368321, 'longitude': -5.581312}

{'name': 'Dragonstone', 'latitude': 55.167801, 'longitude': -6.815096}



The query returns the location name, as well as the coordinate. The coordinate is returned as two separate attributes. You may use a simpler `RETURN loc` instead if you want.

To create the required edge documents to store these relations in the database, we can run a query that combines joining and filtering to match up the right character documents, then use their `_id` attribute to insert an edge into an edge collection ChildOf.

# Find locations within radius

Instead of `NEAR()` we can also use `WITHIN()`, to search for locations within a given radius from a reference point. The syntax is the same as for `NEAR()`, except for the fourth parameter, which specifies the radius instead of a limit. The unit for the radius is meters. The example uses a radius of 200,000 meters (200 kilometers):

In [13]:
within_locations_names = """
FOR loc IN WITHIN(Locations, 53.35, -6.26, 200 * 1000)
    RETURN {
        name: loc.name,
        latitude: loc.coordinate[0],
        longitude: loc.coordinate[1]
    }
"""

query_result = db.AQLQuery(within_locations_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print() 

{'name': 'Vaes Dothrak', 'latitude': 54.16776, 'longitude': -6.096125}

{'name': 'Winterfell', 'latitude': 54.368321, 'longitude': -5.581312}



# Calculating the Distance

Both `NEAR()` and `WITHIN()` can return the distance to the reference point by adding an optional fifth parameter. It has to be a string, which will be used as attribute name for an additional attribute with the distance in meters:

In [14]:
near_locations_names = """
FOR loc IN NEAR(Locations, 53.35, -6.26, 3, "distance")
    RETURN {
        name: loc.name,
        latitude: loc.coordinate[0],
        longitude: loc.coordinate[1],
        distance: loc.distance / 1000
    }
"""

query_result = db.AQLQuery(near_locations_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()    


{'name': 'Vaes Dothrak', 'latitude': 54.16776, 'longitude': -6.096125, 'distance': 91.56658640314484}

{'name': 'Winterfell', 'latitude': 54.368321, 'longitude': -5.581312, 'distance': 121.66399816395003}

{'name': 'Dragonstone', 'latitude': 55.167801, 'longitude': -6.815096, 'distance': 205.31879386198273}



The extra attribute, here called distance, is returned as part of the loc variable, as if it was part of the location document. The value is divided by 1000 in the example query, to convert the unit to kilometers, simply to make it better readable.

# Next Steps

To continue playing and working with ArangoDB beyond the temporary database, you can:

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=AQLJoin&utm_medium=Github&utm_campaign=ArangoDB%20University)
* Take the [free Graph Course](https://www.arangodb.com/arangodb-graph-course)  
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* Keep Learning at thttps://www.arangodb.com/arangodb-training-center/

# Further Links

* https://www.arangodb.com/docs/stable/aql/tutorial.html